Confidence değerleri soccer net ball action dosyası için düzenlendi

In [ ]:
import json

# Load the original data
with open('/content/drive/MyDrive/alperen_merge/ball_results_spotting.json', 'r') as f:
    data = json.load(f)

# Filter predictions with confidence > 0.40
filtered_predictions = [
    p for p in data.get('predictions', [])
    if float(p.get('confidence', 0)) > 0.40
]

# Create new structure
filtered_data = {
    "UrlLocal": data.get("UrlLocal"),
    "predictions": filtered_predictions
}

# Write to filtered_ball.json
with open('/content/drive/MyDrive/alperen_merge/filtered_ball.json', 'w') as f:
    json.dump(filtered_data, f, indent=4)

print(f"✅ 'filtered_ball.json' oluşturuldu. Toplam {len(filtered_predictions)} kayıt yazıldı.")


✅ 'filtered_ball.json' oluşturuldu. Toplam 100 kayıt yazıldı.


Poazisyon tekrarları pass dosyaları için çıkarılıyor

In [ ]:
import json

# 1. Etiketli pozisyonlar dosyasını oku
with open('/content/drive/MyDrive/alperen_merge/etiketli_pozisyonlar.json', 'r', encoding='utf-8') as f:
    etiketli = json.load(f)

# 2. "Pozisyon_Tekrarı_Başlangıcı" ve "Pozisyon_Tekrarı_Bitişi" aralıklarını oluştur
intervals = []
current_start = None
for entry in etiketli:
    mm, ss = entry['gameTime'].split(':')
    t = int(mm) * 60 + int(ss)  # saniyeye çevir
    if entry['label'] == "Pozisyon_Tekrarı_Başlangıcı":
        current_start = t
    elif entry['label'] == "Pozisyon_Tekrarı_Bitişi" and current_start is not None:
        intervals.append((current_start, t))
        current_start = None

# 3. filtered_ball.json dosyasını oku
with open('/content/drive/MyDrive/alperen_merge/filtered_ball.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 4. Zaman stringini saniyeye çeviren yardımcı fonksiyon
def time_to_seconds(t_str):
    mm, ss = t_str.split(':')
    return int(mm) * 60 + int(ss)

# 5. Önerileri (predictions) filtrele
filtered_preds = []
for pred in data.get('predictions', []):
    t = time_to_seconds(pred['gameTime'])
    # Eğer herhangi bir interval içinde ise atla
    if any(start <= t <= end for start, end in intervals):
        continue
    filtered_preds.append(pred)

# 6. Yeni yapıyı hazırla
output_data = {
    "UrlLocal": data.get("UrlLocal"),
    "predictions": filtered_preds
}

# 7. Sonucu filtered_with_etiket.json olarak kaydet
with open('/content/drive/MyDrive/alperen_merge/filtered_ball_with_etiket.json', 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=4)

print(f"✅ İşlem tamamlandı: {len(filtered_preds)} kayıt yazıldı.")




✅ İşlem tamamlandı: 72 kayıt yazıldı.


In [ ]:
import json
import os

# 1. Etiketli pozisyonlar dosyasını oku
etiketli_path = '/content/drive/MyDrive/alperen_merge/etiketli_pozisyonlar.json'
with open(etiketli_path, 'r', encoding='utf-8') as f:
    etiketli = json.load(f)

# 2. "Pozisyon_Tekrarı_Başlangıcı" ve "Pozisyon_Tekrarı_Bitişi" aralıklarını oluştur
intervals = []
current_start = None
for entry in etiketli:
    mm, ss = entry['gameTime'].split(':')
    t = int(mm) * 60 + int(ss)
    if entry['label'] == "Pozisyon_Tekrarı_Başlangıcı":
        current_start = t
    elif entry['label'] == "Pozisyon_Tekrarı_Bitişi" and current_start is not None:
        intervals.append((current_start, t))
        current_start = None

# 3. pass_analysis_simple dosyasını bul ve oku
files = os.listdir('/content/drive/MyDrive/alperen_merge')
pass_file_name = next(f for f in files if 'pass_analysis_simple' in f)
pass_file_path = os.path.join('/content/drive/MyDrive/alperen_merge', pass_file_name)

with open(pass_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 4. Zaman stringini saniyeye çeviren yardımcı fonksiyon
def time_to_seconds(t_str):
    # Örneğin "0:21" ya da "00:21"
    parts = t_str.split(':')
    mm, ss = int(parts[0]), int(parts[1])
    return mm * 60 + ss

# 5. Aralıklardaki kayıtları çıkar
filtered = [
    entry for entry in data
    if not any(start <= time_to_seconds(entry['zaman']) <= end for start, end in intervals)
]

# 6. Sonucu filtered_pass_analysis.json olarak kaydet
output_path = '/content/drive/MyDrive/alperen_merge/filtered_pass_analysis.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(filtered, f, indent=4)

print(f"✅ İşlem tamamlandı: {len(filtered)} kayıt '{output_path}' dosyasına kaydedildi.")



✅ İşlem tamamlandı: 49 kayıt '/content/drive/MyDrive/alperen_merge/filtered_pass_analysis.json' dosyasına kaydedildi.


Indirect Free Kick filtereleniyor

In [ ]:
import json
import glob
import os

def time_to_seconds(t_str):
    """Convert 'MM:SS' or 'M:SS' string to total seconds."""
    mm, ss = t_str.split(':')
    return int(mm) * 60 + int(ss)

# 1. Locate and load the three input files
events_file = glob.glob('/content/drive/MyDrive/alperen_merge/events_without_goals.json')[0]
fwe_file    = '/content/drive/MyDrive/alperen_merge/filtered_with_etiket.json'
fpa_file    = '/content/drive/MyDrive/alperen_merge/filtered_pass_analysis.json'

with open(events_file, 'r', encoding='utf-8') as f:
    events = json.load(f)               # list of {gameTime, label, confidence}

with open(fwe_file, 'r', encoding='utf-8') as f:
    fwe = json.load(f)                  # dict with 'UrlLocal' and 'predictions'
fwe_preds = fwe['predictions']

with open(fpa_file, 'r', encoding='utf-8') as f:
    fpa = json.load(f)                  # list of {zaman, frame, …}

# 2. Prepare output copies
last_events = list(events)
last_fwe_preds = list(fwe_preds)
last_fpa = list(fpa)

# 3. Process each "Indirect free-kick" in the events list
for ev in events:
    if ev.get('label') != 'Indirect free-kick':
        continue

    t = time_to_seconds(ev['gameTime'])
    start, end = t - 2, t + 2

    # Count nearby events in each filtered file
    count_fwe = sum(1 for p in fwe_preds if start <= time_to_seconds(p['gameTime']) <= end)
    count_fpa = sum(1 for p in fpa   if start <= time_to_seconds(p['zaman'])    <= end)

    if count_fwe > 0 and count_fpa > 0:
        # Both have events: remove this Indirect free-kick
        last_events = [
            e for e in last_events
            if not (e.get('label') == 'Indirect free-kick'
                    and time_to_seconds(e['gameTime']) == t)
        ]
    else:
        # At least one file lacks events: remove any events in that interval
        last_fwe_preds = [
            p for p in last_fwe_preds
            if not (start <= time_to_seconds(p['gameTime']) <= end)
        ]
        last_fpa = [
            p for p in last_fpa
            if not (start <= time_to_seconds(p['zaman']) <= end)
        ]

# 4. Write out the three resulting files
out_dir = '/mnt/data'
with open(os.path.join(out_dir, '/content/drive/MyDrive/alperen_merge/last_events_without_goals.json'), 'w', encoding='utf-8') as f:
    json.dump(last_events, f, indent=4)

with open(os.path.join(out_dir, '/content/drive/MyDrive/alperen_merge/last_filtered_with_etiket.json'), 'w', encoding='utf-8') as f:
    json.dump({
        'UrlLocal': fwe['UrlLocal'],
        'predictions': last_fwe_preds
    }, f, indent=4)

with open(os.path.join(out_dir, '/content/drive/MyDrive/alperen_merge/last_filtered_pass_analysis.json'), 'w', encoding='utf-8') as f:
    json.dump(last_fpa, f, indent=4)

print("✅ Oluşturuldu:\n"
      "- last_events_without_goals.json\n"
      "- last_filtered_with_etiket.json\n"
      "- last_filtered_pass_analysis.json")


✅ Oluşturuldu:
- last_events_without_goals.json
- last_filtered_with_etiket.json
- last_filtered_pass_analysis.json


Direct Free Kick

In [ ]:
import json
import os

def time_to_seconds(t_str):
    """Convert 'MM:SS' or 'M:SS' string to total seconds."""
    mm, ss = t_str.split(':')
    return int(mm) * 60 + int(ss)

# 1. Load events_without_goals.json
files = os.listdir('/content/drive/MyDrive/alperen_merge/')
events_filename = next(f for f in files if 'last_events_without_goals.json' in f)
with open(os.path.join('/content/drive/MyDrive/alperen_merge/', events_filename), 'r', encoding='utf-8') as f:
    events = json.load(f)  # List of {gameTime, label, confidence}

# 2. Load filtered_with_etiket.json
fwe_path = '/content/drive/MyDrive/alperen_merge/last_filtered_with_etiket.json'
with open(fwe_path, 'r', encoding='utf-8') as f:
    fwe_data = json.load(f)
fwe_preds = fwe_data['predictions']

# 3. Load filtered_pass_analysis.json
fpa_path = '/content/drive/MyDrive/alperen_merge/last_filtered_pass_analysis.json'
with open(fpa_path, 'r', encoding='utf-8') as f:
    fpa_preds = json.load(f)  # List of dicts with key 'zaman'

# 4. Process each "Direct free-kick"
for ev in events:
    if ev.get('label') != 'Direct free-kick':
        continue

    t = time_to_seconds(ev['gameTime'])
    start, end = t - 3, t + 3

    # Count events in each list within ±2 seconds
    count_fwe = sum(1 for p in fwe_preds if start <= time_to_seconds(p['gameTime']) <= end)
    count_fpa = sum(1 for p in fpa_preds if start <= time_to_seconds(p['zaman']) <= end)

    # If both have at least one event in that window, remove those events
    if count_fwe > 0 and count_fpa > 0:
        fwe_preds = [
            p for p in fwe_preds
            if not (start <= time_to_seconds(p['gameTime']) <= end)
        ]
        fpa_preds = [
            p for p in fpa_preds
            if not (start <= time_to_seconds(p['zaman']) <= end)
        ]

# 5. Write out the results
out_dir = '/content/drive/MyDrive/alperen_merge'
with open(os.path.join(out_dir, '/content/drive/MyDrive/alperen_merge/2_last_filtered_with_etiket.json'), 'w', encoding='utf-8') as f:
    json.dump({'UrlLocal': fwe_data.get('UrlLocal'), 'predictions': fwe_preds}, f, indent=4)

with open(os.path.join(out_dir, '/content/drive/MyDrive/alperen_merge/2_last_filtered_pass_analysis.json'), 'w', encoding='utf-8') as f:
    json.dump(fpa_preds, f, indent=4)

print("✅ Created files:\n"
      "- last_filtered_with_etiket2.json\n"
      "- last_filtered_pass_analysis2.json")


✅ Created files:
- last_filtered_with_etiket2.json
- last_filtered_pass_analysis2.json


Filtered Events pozisyon tekrarları çıkarılıyor

In [ ]:
import json
import os

# 1. Etiketli pozisyonlar dosyasını oku
etiketli_path = '/content/drive/MyDrive/alperen_merge/etiketli_pozisyonlar.json'
with open(etiketli_path, 'r', encoding='utf-8') as f:
    etiketli = json.load(f)

# 2. "Pozisyon_Tekrarı_Başlangıcı" ve "Pozisyon_Tekrarı_Bitişi" aralıklarını oluştur
intervals = []
current_start = None
for entry in etiketli:
    mm, ss = entry['gameTime'].split(':')
    t = int(mm) * 60 + int(ss)
    if entry['label'] == "Pozisyon_Tekrarı_Başlangıcı":
        current_start = t
    elif entry['label'] == "Pozisyon_Tekrarı_Bitişi" and current_start is not None:
        intervals.append((current_start, t))
        current_start = None

# 3. pass_analysis_simple dosyasını bul ve oku
files = os.listdir('/content/drive/MyDrive/alperen_merge')
pass_file_name = next(f for f in files if 'last_events_without_goals' in f)
pass_file_path = os.path.join('/content/drive/MyDrive/alperen_merge', pass_file_name)

with open(pass_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 4. Zaman stringini saniyeye çeviren yardımcı fonksiyon
def time_to_seconds(t_str):
    # Örneğin "0:21" ya da "00:21"
    parts = t_str.split(':')
    mm, ss = int(parts[0]), int(parts[1])
    return mm * 60 + ss

# 5. Aralıklardaki kayıtları çıkar
filtered = [
    entry for entry in data
    if not any(start < time_to_seconds(entry['gameTime']) < end for start, end in intervals)
]

# 6. Sonucu filtered_pass_analysis.json olarak kaydet
output_path = '/content/drive/MyDrive/alperen_merge/2_last_events_without_goals.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(filtered, f, indent=4)

print(f"✅ İşlem tamamlandı: {len(filtered)} kayıt '{output_path}' dosyasına kaydedildi.")

✅ İşlem tamamlandı: 26 kayıt '/content/drive/MyDrive/alperen_merge/2_last_events_without_goals.json' dosyasına kaydedildi.


Eventlerde Shots-On target ve Shots off Target kısımlarındaki verilerin gol olduğu zaman silinmesi

In [ ]:
import json
import pandas as pd
import os

def time_to_seconds(t_str):
    """Convert 'MM:SS' or 'M:SS' string to total seconds."""
    mm, ss = t_str.split(':')
    return int(mm) * 60 + int(ss)

# Dosya yolları
data_dir = '/content/drive/MyDrive/alperen_merge'
events_path = os.path.join(data_dir, '2_last_events_without_goals.json')
goals_csv_path = os.path.join(data_dir, 'verified_goals.csv')
output_path = os.path.join(data_dir, '2_events_without_goals.json')

# 1. Gol zamanlarını oku
goals_df = pd.read_csv(goals_csv_path)  # sütunlar: 'ozet_dakika', 'ozet_saniye'
goals_seconds = (goals_df['ozet_dakika'] * 60 + goals_df['ozet_saniye']).tolist()

# 2. Olayları yükle
with open(events_path, 'r', encoding='utf-8') as f:
    events = json.load(f)

# 3. Filtreleme: ±2 saniye içinde olanları çıkar
filtered_events = [
    ev for ev in events
    if not any(abs(time_to_seconds(ev['gameTime']) - gs) <= 2 for gs in goals_seconds)
]

# 4. Sonucu kaydet
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(filtered_events, f, indent=4)

print(f"✅ {len(events) - len(filtered_events)} kayıt silindi, geriye {len(filtered_events)} kaldı.")
print(f"Filtrelenmiş dosya: {output_path}")


✅ 5 kayıt silindi, geriye 21 kaldı.
Filtrelenmiş dosya: /content/drive/MyDrive/alperen_merge/2_events_without_goals.json


Shots On ve Shots off labellarını düzenliyorum

In [ ]:
import json
import os

def time_to_seconds(t_str):
    """Convert 'MM:SS' or 'M:SS' to total seconds (float)."""
    mm, ss = t_str.split(':')
    return int(mm) * 60 + float(ss)

def seconds_to_time(total_seconds):
    """Convert seconds (float) back to 'MM:SS' string with rounding."""
    mm = int(total_seconds // 60)
    ss = int(round(total_seconds - mm * 60))
    return f"{mm:02d}:{ss:02d}"

# 1. Load events
input_path = '/content/drive/MyDrive/alperen_merge/2_events_without_goals.json'
with open(input_path, 'r', encoding='utf-8') as f:
    events = json.load(f)  # list of dicts

paired = set()      # indices of events to remove
new_events = []     # list for the merged events

# 2. Find and merge Shots on/off target pairs
for i, ev in enumerate(events):
    if i in paired:
        continue
    if ev.get('label') == 'Shots on target':
        t_on = time_to_seconds(ev['gameTime'])
        # search for off-target within ±3s
        for j, ev2 in enumerate(events):
            if j in paired or ev2.get('label') != 'Shots off target':
                continue
            t_off = time_to_seconds(ev2['gameTime'])
            if abs(t_on - t_off) <= 3:
                # mark both for removal
                paired.update([i, j])
                # create merged event
                avg_time = (t_on + t_off) / 2
                avg_conf = (float(ev['confidence']) + float(ev2['confidence'])) / 2
                new_ev = {
                    "gameTime": seconds_to_time(avg_time),
                    "label": "Critic goal position",
                    "confidence": f"{avg_conf:.6f}"
                }
                new_events.append(new_ev)
                break

# 3. Build the final list: keep unpaired + add new events
final_events = [
    ev for idx, ev in enumerate(events) if idx not in paired
] + new_events

# 4. Sort by time
final_events.sort(key=lambda e: time_to_seconds(e['gameTime']))

# 5. Write to output file
output_path = '/content/drive/MyDrive/alperen_merge/3_events_without_goals.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(final_events, f, indent=4)

print(f"✅ Processed events written to {output_path} "
      f"({len(paired)//2} pairs merged into {len(new_events)} new events).")


✅ Processed events written to /content/drive/MyDrive/alperen_merge/3_events_without_goals.json (3 pairs merged into 3 new events).


Pozisyonlarda hangi takım hücum yapıyor ona bakılıyor

In [ ]:
import json
import pandas as pd

def time_to_seconds(t_str):
    """Convert 'MM:SS' or 'M:SS' to total seconds."""
    mm, ss = t_str.split(':')
    return int(mm) * 60 + int(ss)

# 1. etiketli_pozisyonlar dosyasını oku
with open('/content/drive/MyDrive/alperen_merge/etiketli_pozisyonlar.json', 'r', encoding='utf-8') as f:
    pozisyonlar = json.load(f)

# 2. "Pozisyon_Başlangıcı"–"Pozisyon_Bitişi" aralıklarını ve başlangıç indekslerini oluştur
intervals = []
current_start = None
current_idx = None
for idx, ev in enumerate(pozisyonlar):
    t = time_to_seconds(ev['gameTime'])
    if ev['label'] == 'Pozisyon_Başlangıcı':
        current_start = t
        current_idx = idx
    elif ev['label'] == 'Pozisyon_Bitişi' and current_start is not None:
        intervals.append((current_start, t, current_idx))
        current_start = None
        current_idx = None

# 3. verified_goals.csv dosyasını oku
df = pd.read_csv('/content/drive/MyDrive/alperen_merge/verified_goals.csv')  # sütunlar: ozet_dakika, ozet_saniye, takim, vb.
df['goal_sec'] = df['ozet_dakika'] * 60 + df['ozet_saniye']

# 4. Her gol için ilgili aralığı bul ve atak_yapan_takım ekle
for _, row in df.iterrows():
    sec = int(row['goal_sec'])
    team = row['takim']
    for start, end, idx in intervals:
        if start <= sec <= end:
            ev = pozisyonlar[idx]
            # Eğer daha önce eklenmişse listeye ekle, değilse doğrudan ata
            if 'atak_yapan_takım' in ev:
                existing = ev['atak_yapan_takım']
                if isinstance(existing, list):
                    existing.append(team)
                else:
                    ev['atak_yapan_takım'] = [existing, team]
            else:
                ev['atak_yapan_takım'] = team
            break

# 5. Yeni dosyayı kaydet
with open('/content/drive/MyDrive/alperen_merge/atak_etiketli_pozisyonlar.json', 'w', encoding='utf-8') as f:
    json.dump(pozisyonlar, f, ensure_ascii=False, indent=4)

print("✅ 'atak_etiketli_pozisyonlar.json' oluşturuldu.")


✅ 'atak_etiketli_pozisyonlar.json' oluşturuldu.


Atak Etiketli Pozisyonlara ekstra olarak betülün pass dosyasının ekleyen kod

In [ ]:
import json
import os

def time_to_seconds(t_str):
    """Convert 'MM:SS' to total seconds."""
    mm, ss = t_str.split(':')
    return int(mm) * 60 + int(ss)

# Dosya yolları
data_dir = '/content/drive/MyDrive/alperen_merge'
pos_file = os.path.join(data_dir, 'atak_etiketli_pozisyonlar.json')
pass_analysis_file = os.path.join(data_dir, '2_last_filtered_pass_analysis.json')
output_file = os.path.join(data_dir, '2_atak_etiketli_pozisyonlar.json')

# 1. Pozisyon verisini yükle
with open(pos_file, 'r', encoding='utf-8') as f:
    pozisyonlar = json.load(f)

# 2. Başlangıç ve bitiş aralıklarını yakala
intervals = []
start_idx = None
start_time = None
for idx, ev in enumerate(pozisyonlar):
    t = time_to_seconds(ev['gameTime'])
    if ev['label'] == 'Pozisyon_Başlangıcı':
        start_time, start_idx = t, idx
    elif ev['label'] == 'Pozisyon_Bitişi' and start_time is not None:
        intervals.append((start_time, t, start_idx))
        start_time, start_idx = None, None

# 3. Pass analizi verisini yükle
with open(pass_analysis_file, 'r', encoding='utf-8') as f:
    pass_events = json.load(f)

# 4. Her interval için sayım ve atama
for start, end, idx in intervals:
    ev = pozisyonlar[idx]
    if 'atak_yapan_takım' in ev:
        continue

    counts = {0: 0, 1: 0}
    for pe in pass_events:
        t_pe = time_to_seconds(pe['zaman'])
        if start <= t_pe <= end:
            team = pe.get('topa_sahip_takım')
            if team in counts:
                counts[team] += 1

    if counts[0] > counts[1]:
        ev['atak_yapan_takım'] = 0
    elif counts[1] > counts[0]:
        ev['atak_yapan_takım'] = 1
    else:
        ev['atak_yapan_takım'] = None

# 5. Güncellenmiş JSON'u kaydet
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(pozisyonlar, f, ensure_ascii=False, indent=4)

print(f"✅ Güncellenmiş dosya: {output_file}")


✅ Güncellenmiş dosya: /content/drive/MyDrive/alperen_merge/2_atak_etiketli_pozisyonlar.json


PASS Analysis dosyasını güncelliyorum.

In [ ]:
import json
import os

def time_to_seconds(t_str):
    """Convert 'MM:SS' or 'M:SS' to total seconds."""
    mm, ss = t_str.split(':')
    return int(mm) * 60 + int(ss)

# Paths
data_dir = '/content/drive/MyDrive/alperen_merge/'
events_file = os.path.join(data_dir, '3_events_without_goals.json')
pass_analysis_file = os.path.join(data_dir, '2_last_filtered_pass_analysis.json')
output_file = os.path.join(data_dir, 'son_last_filtered_past_analysis.json')

# Load events without goals
with open(events_file, 'r', encoding='utf-8') as f:
    events = json.load(f)

# Load pass analysis data
with open(pass_analysis_file, 'r', encoding='utf-8') as f:
    pass_events = json.load(f)

# Remove pass_events within ±1 second of any event gameTime
filtered_pass_events = pass_events
for ev in events:
    t = time_to_seconds(ev['gameTime'])
    start, end = t - 1, t + 1
    filtered_pass_events = [
        pe for pe in filtered_pass_events
        if not (start <= time_to_seconds(pe['zaman']) <= end)
    ]

# Write filtered results
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(filtered_pass_events, f, indent=4)

print(f"✅ Filtered pass analysis events written to {output_file}")


✅ Filtered pass analysis events written to /content/drive/MyDrive/alperen_merge/son_last_filtered_past_analysis.json


PAST ETİKET DOSYASINI Güncelliyorum

In [ ]:
import json
import os

def time_to_seconds(t_str):
    """Convert 'MM:SS' or 'M:SS' to total seconds."""
    mm, ss = t_str.split(':')
    return int(mm) * 60 + int(ss)

# Dosya yolları
data_dir = '/content/drive/MyDrive/alperen_merge/'
events_file = os.path.join(data_dir, '3_events_without_goals.json')
with_etiket_file = os.path.join(data_dir, '2_last_filtered_with_etiket.json')
output_file = os.path.join(data_dir, 'son_last_filtered_with_etiket.json')

# 1. Olayları yükle
with open(events_file, 'r', encoding='utf-8') as f:
    events = json.load(f)

# 2. filtered_with_etiket dosyasını yükle
with open(with_etiket_file, 'r', encoding='utf-8') as f:
    data = json.load(f)
preds = data.get('predictions', [])

# 3. Her event için ±1 saniyelik aralıktaki kayıtları çıkar
for ev in events:
    t = time_to_seconds(ev['gameTime'])
    start, end = t - 1, t + 1
    preds = [
        p for p in preds
        if not (start <= time_to_seconds(p['gameTime']) <= end)
    ]

# 4. Sonucu kaydet
result = {
    'UrlLocal': data.get('UrlLocal'),
    'predictions': preds
}
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

print(f"✅ Çıktı: {output_file}")



✅ Çıktı: /content/drive/MyDrive/alperen_merge/son_last_filtered_with_etiket.json


İki dosyayı merge ediyoruz

In [ ]:
import json
import os

def time_to_seconds(t_str):
    """Convert 'MM:SS' or 'M:SS' to total seconds."""
    parts = t_str.split(':')
    minutes = int(parts[0])
    seconds = int(parts[1])
    return minutes * 60 + seconds

# File paths
data_dir = '/content/drive/MyDrive/alperen_merge'
past_analysis_path = os.path.join(data_dir, 'son_last_filtered_past_analysis.json')
with_etiket_path  = os.path.join(data_dir, 'son_last_filtered_with_etiket.json')
output_path       = os.path.join(data_dir, 'merged_pass.json')

# Load the past analysis events
with open(past_analysis_path, 'r', encoding='utf-8') as f:
    past_events = json.load(f)

# Load the predictions with etikets
with open(with_etiket_path, 'r', encoding='utf-8') as f:
    with_data = json.load(f)
predictions = with_data.get('predictions', [])

# Build a set of times (in seconds) where a top_degisimi occurred in past analysis
top_degisimi_times = {
    time_to_seconds(ev['zaman'])
    for ev in past_events
    if ev.get('event_type') == 'top_degisimi'
}

# Traverse predictions and update labels
for p in predictions:
    pt = time_to_seconds(p['gameTime'])
    if pt in top_degisimi_times and p.get('label') == 'PASS':
        # confidence is a string, convert to float
        if float(p.get('confidence', 0)) < 0.55:
            p['label'] = 'top_degisimi'

# Write merged results
merged = {
    'UrlLocal': with_data.get('UrlLocal'),
    'predictions': predictions
}
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(merged, f, ensure_ascii=False, indent=4)

print(f"✅ 'merged_pass.json' oluşturuldu: {output_path}")


✅ 'merged_pass.json' oluşturuldu: /content/drive/MyDrive/alperen_merge/merged_pass.json


Goller csv dosyasından jsona dönüyor

In [ ]:
import pandas as pd
import json
import os

# 1. CSV dosyasının yolu
csv_path = '/content/drive/MyDrive/alperen_merge/verified_goals.csv'  # Dosya adını gerektiği gibi güncelle

# 2. CSV'yi oku
df = pd.read_csv(csv_path)

# 3. 'real_minute' sütununu kaldır (varsa)
df = df.drop(columns=['real_minute'], errors='ignore')

# 4. 'ozet_dakika' ve 'ozet_saniye' sütunlarını birleştirerek 'gameTime' oluştur
df['gameTime'] = df.apply(
    lambda row: f"{int(row['ozet_dakika']):02d}:{int(row['ozet_saniye']):02d}", axis=1
)

# 5. Orijinal sütunları çıkar
df = df.drop(columns=['ozet_dakika', 'ozet_saniye'])

# 6. JSON formatına dönüştür
records = df.to_dict(orient='records')

# 7. JSON'u dosyaya kaydet
output_path = '/content/drive/MyDrive/alperen_merge/verified_goals.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(records, f, ensure_ascii=False, indent=4)

print(f"✅ CSV başarıyla JSON'a dönüştürüldü: {output_path}")


✅ CSV başarıyla JSON'a dönüştürüldü: /content/drive/MyDrive/alperen_merge/verified_goals.json


ATAK ETİKETLİ POZİSYONLARA BEŞİKTAŞ VE ANTALYA OLARAK ATANIYOR

In [ ]:
import json
import os

# 1. Girdi ve çıktı dosya yolları
input_path = '/content/drive/MyDrive/alperen_merge/2_atak_etiketli_pozisyonlar.json'
output_path = '/content/drive/MyDrive/alperen_merge/son_atak_etiketli_pozisyonlar.json'

# 2. JSON'u oku
with open(input_path, 'r', encoding='utf-8') as f:
    pozisyonlar = json.load(f)

# 3. Sayısal atak_yapan_takım değerlerini isimlere çevir
team_map = {
    0: "BITEXEN ANTALYASPOR",
    1: "BEŞİKTAŞ A.Ş."
}

for ev in pozisyonlar:
    if 'atak_yapan_takım' in ev and isinstance(ev['atak_yapan_takım'], int):
        ev['atak_yapan_takım'] = team_map.get(ev['atak_yapan_takım'], ev['atak_yapan_takım'])

# 4. Güncellenmiş veriyi kaydet
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(pozisyonlar, f, ensure_ascii=False, indent=4)

print(f"✅ '{output_path}' dosyası oluşturuldu.")


✅ '/content/drive/MyDrive/alperen_merge/son_atak_etiketli_pozisyonlar.json' dosyası oluşturuldu.
